# Environment setup

Sets Together AI API key for this session.

In [ ]:
import os

TOGETHER_API_KEY = "tgp_v1_eK-miWGtvpUsYhRdy9w55WavGtmHrjzvJqC2ui-5PaM"

os.environ["TOGETHER_API_KEY"] = TOGETHER_API_KEY
os.environ["OPENAI_API_KEY"] = TOGETHER_API_KEY  # reused by LangChain ChatOpenAI
print("Together API key set for this session.")


In [ ]:
from features.features import BankingTransactionQA

together_key = os.environ.get("TOGETHER_API_KEY")
qa = BankingTransactionQA(
    uri="tcp://7.tcp.eu.ngrok.io:10126",
    username="neo4j",
    password="12345678",
    database="transactionsnew",
    llm_provider="openai",
    api_key=together_key,
    temperature=0,
)
